Setup

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

데이터 불러오기

In [20]:
import os
import tarfile
import urllib

In [21]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [22]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [23]:
housing = load_housing_data()
# 데이터 앞 5개 보기
housing.head()

NameError: name 'housing' is not defined

In [ ]:
# 데이터 정보 요약
housing.info()

In [ ]:
# total_bedrooms의 결측치 존재를 확인
# ocean_proximity의 자료형이 object임을 확인
# ocean_proximity의 내용을 봐선 categorical(범주형)자료일 것

In [ ]:
# ocean_proximity의 카테고리를 확인
housing["ocean_proximity"].value_counts()

In [ ]:
# 다른 숫자형 필드의 특징 요약
housing.describe()

In [ ]:
# 히스토그램 그래프형으로 보기
import matplotlib as plt
housing.hist(bins=50, figsize=(20, 15))

In [ ]:
# 1. median_income이 dollar단위가 아닌 것으로 보임. x 10000한 값이 실제 dollar인 것으로 확인
# 2. housing_median_age와 median_house_value가 최대 최소값을 한정해 놓음(그래프 끝이 비정상적으로 올라감). 처리는 정확한 값을 구하거나 이 값을 제거하는 것
# 3. 특성들간 스케일이 많이 다름
# 4. 많은 히스토그램들의 꼬리가 두꺼움(중심에서 왼쪽보다 오른쪽으로 더 멀리 뻗어 있음). 이는 알고리즘이 패턴을 찾기 어렵게함. 종 모양 분포로 변형 필요.

In [ ]:
# 테스트 데이터 세트 만들기
import numpy as np

np.random.seed(42)

def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
train_set, test_set = split_train_test(housing, 0.2)

In [ ]:
# 다른 방법
from sklearn.model_selection import train_test_split

train_set_2, test_set_2 = train_test_split(housing, test_size=0.2, random_state=42)

In [ ]:
# 완전무작위로 데이터를 나누는 것은 샘플링 편향이 생길 가능성이 크다.
# 소득의 비율별로 데이터를 나눠보자.
# 소득을 구간별로 나눠 카테고리화 하기. 5구간.
housing["income_cat"] = pd.cut(housing["median_income"],
                              bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                              labels=[1, 2, 3, 4, 5])

housing["income_cat"].hist()

In [ ]:
# 소득 카테고리를 기반으로 계층 샘플링하기
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

In [ ]:
# income_cat 삭제해 원래대로 되돌리기
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [ ]:
# 데이터가 잘 분리되었으니 이제 훈련 세트만 보자. 훈련세트 손상을 방지하기 위해 복사본을 만들기.
housing = strat_train_set.copy()

In [ ]:
# 지리 정보(위도, 경도)를 시각화해보자. 점의 밀집도를 알기 위해 alpha=0.1로 점이 몰리는 곳이 진해지게 함.
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

In [ ]:
# 더 많은 정보를 나타내 보자. 원 크기가 인구, 원 색이 집값.
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
            s=housing["population"]/100, label="population", figsize=(10,7),
            c="median_house_value", cmap=plt.pyplot.get_cmap("jet"), colorbar=True,
            sharex=False)

In [ ]:
# 모든 특성 간의 표준 상관계수 보기. #1이면 강한 양의 상관관계, -1이면 강한 음의 상관관계, 0이면 어떤 선형적 상관관계도 없음.
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
# 상관관계를 확인하는 다른 방법인 산점도를 median_house_income과 관련있어 보이는 특성들과 살펴보기.

attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
pd.plotting.scatter_matrix(housing[attributes], figsize=(12, 8))

In [ ]:
# 가장 상관관계가 높은 두 특성을 확대.
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.1)

In [ ]:
# 보면 500000, 450000, 350000, 280000과 그 아래에 직선 형태를 볼 수 있다. 이것은 제거 필요.

In [ ]:
# 방 개수, 전체 침실수, 인구는 그 자체로는 별 쓸모 없다. 이들을 쓸모있어 보이는 특성으로 바꿔보자.
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"] = housing["population"]/housing["households"]

In [ ]:
# 상관관계를 다시 파악해보자.
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
# 본격적으로 알고리즘을 위한 데이터를 준비해보자.
# 원래 훈련 세트로 복원하고, 예측 변수와 타깃값에 같은 변형이 가지 않도록 레이블을 분리하자.
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

In [ ]:
# total_bedrooms의 결측치 다루기
# 1. 해당 구역 제거
#housing.dropna(subset=["total_bedrooms"])
# 2. 전체 특성 삭제
#housing.drop("total_bedrooms", axis=1)
# 3. 어떤 값으로 채우기(중간값. 여기서 계산된 중간값 저장해놔야 함.)
#median = housing["total_bedrooms"].median()
#housing["total_bedrooms"],fillna(median, inplace=True)

In [ ]:
# 결측치 다루는 다른 방법
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")
# 중간값은 수치형 특성에만 적용되므로 텍스트 특성인 ocean_proximity 제거한 복사본으로 진행. 다른 특성엔 결측치가 없지만 나중에 새로운 데이터 추가될 때 생길 수 있으니 한꺼번에 하는게 바랍직.
housing_num = housing.drop("ocean_proximity", axis=1)
imputer.fit(housing_num)

In [ ]:
# imputer이 계산한 중간값은 객체의 statistics_ 속성에 저장됨. 나중에 사용 가능
imputer.statistics_

In [ ]:
housing_num.median().values

In [ ]:
# 결측치에 적용시키기
X = imputer.transform(housing_num)
housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)

In [ ]:
# 텍스트, 범주형 특성 다루기(여기선 ocean_proximity)
# 첫 10 샘플에서 특성값 확인.
housing_cat = housing[["ocean_proximity"]]
housing_cat.head(10)

In [ ]:
# 대부분의 머신러닝 알고리즘은 숫자를 다루므로 이 카테고리를 숫자로 바꿔보자.
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

In [ ]:
# 카테고리 목록은 다음과 같다.
ordinal_encoder.categories_

In [ ]:
# 이 표현 방식의 문제점은 알고리즘이 떨어진 값보다 가까운 값들이 더 비슷하다 생각한다는 것이다.
# 이 문제는 원핫 인고딩으로 해결할 수 있다. 다음과 같다.
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

In [ ]:
# 출력이 SciPy sparse matrix인데 넘파이 배열로 바꾸려면
housing_cat_1hot.toarray()

In [ ]:
# 카테고리 리스트 얻으려면
cat_encoder.categories_

스케일링 하자.

대부분의 알고리즘은 입력 숫자 특성들의 스케일이 많이 다르면 잘 작동하지 않는다.

물론 타깃값은 스케일링할 필요 없다.

min-max 스케일링: 최소값 0, 최대값 1(범위는 지정 가능)로 바꾸는 것

표준화: 평균을 빼고 표준편차로 나누어 결과 분포의 분산이 1이 되도록 함. 이상치에 강함.

(텍스트 쓰는법: 위의 code를 markdown으로 바꿈)

In [ ]:
# 표준화 해보자.
# 우선 뭔지모를 사용자 정의 변환기를 만들어주고
from sklearn.base import BaseEstimator, TransformerMixin

# column index
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)




In [ ]:
# 나머지도 그대로 따라해주고
col_names = "total_rooms", "total_bedrooms", "population", "households"
rooms_ix, bedrooms_ix, population_ix, households_ix = [
    housing.columns.get_loc(c) for c in col_names] # get the column indices

In [ ]:
housing_extra_attribs = pd.DataFrame(
    housing_extra_attribs,
    columns=list(housing.columns)+["rooms_per_household", "population_per_household"],
    index=housing.index)
housing_extra_attribs.head()

In [5]:
# 표준화
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

NameError: name 'SimpleImputer' is not defined

In [ ]:
housing_num_tr

변환기 하나로 각 열마다 변환 적용해주는 더 편한 기능을 사용해보자

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)

In [ ]:
housing_prepared

In [ ]:
housing_prepared.shape

모델 선택하고 훈련해보기

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
# let's try the full preprocessing pipeline on a few training instances
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("Predictions:", lin_reg.predict(some_data_prepared))

In [ ]:
print("Labels:", list(some_labels))

RMSE를 측정해보자.

In [ ]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

예측 오차가 크다. 만족스럽지 못함. 모델이 과소적합된 것으로 보임. 과소적합은 특성들이 충분한 정보를 제공하지 못했거나 모델이 강력하지 않은 것. 과소적합 해결 방법은 더 강력한 모델 사용, 더 좋은 특성 입력, 모델의 규제를 감소시킴임. 이 모델에선 규제를 사용하지 않았으니 규제는 제외. 더 강력한 모델로 훈련해보자.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

과대적합됨.  교차검증으로 평가해보자.

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [6]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

NameError: name 'tree_rmse_scores' is not defined

In [7]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

NameError: name 'cross_val_score' is not defined

결과가 상당히 좋지 않음. 모델 하나 더 시도해보자.

In [8]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(housing_prepared, housing_labels)

NameError: name 'housing_prepared' is not defined

In [ ]:
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

훌륭하지만, 여전히 과대적합된 것으로 보임. 과대적합은 모델을 간단히 하기, 제한(규제)하기, 더 많은 훈련 데이터 모으기로 해결할 수 있음.

모델 세부 튜닝하기

1. 하이퍼파라미터 조절

1-1. GridSearchCV는 사용자가 입력한 각 파라미터의 가능한 값들 중 최적의 값 조합을 일일이 학습하보며 찾음.

In [9]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)

NameError: name 'housing_prepared' is not defined

In [10]:
grid_search.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [11]:
grid_search.best_estimator_

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [12]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'

1-2. RandomizedSearchCV는 랜덤한 파라미터값을 대입해보며 최적의 조합을 찾음. 몇 번 해볼지 조절 가능.

In [13]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    }

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(housing_prepared, housing_labels)

NameError: name 'housing_prepared' is not defined

In [14]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

AttributeError: 'RandomizedSearchCV' object has no attribute 'cv_results_'

이외에 

2. 앙상블

3. 최상의 모델과 오차 분석(상대적 줃요도를 보고 덜 중요한 특성들 제거, 특정 오차를 만든다면 원인 분석후 해결(추가 특성 포함시키기, 불필요한 특성 제거하기, 이상치 제외하기 등))

In [15]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [16]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
#cat_encoder = cat_pipeline.named_steps["cat_encoder"] # old solution
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

NameError: name 'full_pipeline' is not defined

4. 테스트 세트로 시스템 평가하기

In [17]:
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [18]:
final_rmse

NameError: name 'final_rmse' is not defined

95% 신뢰구간 계산하기

In [19]:
from scipy import stats

confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                         loc=squared_errors.mean(),
                         scale=stats.sem(squared_errors)))

NameError: name 'final_predictions' is not defined

하이퍼파라미터 튜닝을 많이 했다면 테스트 세트의 결과가 교차 검증 때보다 조금 낮은 게 보통임(과대적합)

이렇게 해서 좋은 모델이 만들어지면, 론칭하면 됨. 이후 모니터링, 시스템 유지 보수 필요.